# Search Finding

In [ ]:
import os, sys
import json
import requests
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
load_dotenv()

In [ ]:
AUTH = [os.getenv("apiKey"), os.getenv("apiSecret")]
BASE_URL = os.getenv("dome9URL")
HEADER = {
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}

## 0. Search Finding Function

Maximum Total Response 1000 Objects

In [ ]:
def checkFindingsTotal(data={}):
    if data["totalFindingsCount"] == 0:
        print("[Error] Total Finding Counts : 0")

    if data["totalFindingsCount"] == len(data["findings"]):
        print("\nTotal Finding Counts : " + str(data["totalFindingsCount"]))
        return data["findings"]
    else:
        print("[Error] Total Finding Counts != Length of Findings")
        print("[Error] Total Finding Counts : " + str(data["totalFindingsCount"]))
        print("[Error] Length of Findings : " + str(len(data["findings"])))

In [ ]:
def search(filters={}):
    print("[Request] Search Compliance Finding")
    url = BASE_URL + '/Compliance/Finding/search'
    print("URL  : " + url)
    print("Maximum Total Response 1000 Objects")
    
    body = {
        "pageSize": 1000,
        "filter": filters
    }
    
    r = requests.post(url, params={}, data=json.dumps(body).encode("utf-8"), headers=HEADER, auth=(AUTH[0], AUTH[1]))
    if r.status_code == 201:
        r = r.json()
        return checkFindingsTotal(r)
    else:
        print("[Error] " + str(r.status_code))
        print("[Error] " + str(r.text))
        sys.exit()

In [ ]:
def extractFindings(data={}):
    return {
        "cloudAccountId": data["cloudAccountId"],
        "bundleName": data["bundleName"],
        "entityType": data["entityType"],
        "entityName": data["entityName"],
        "ruleName": data["ruleName"],
        "severity": data["severity"],
        "tag": data["tag"],
        "gsl": data["ruleLogic"],
        "remediation": data["remediation"]
    }

## 1. Search Finding by Free Text

In [ ]:
text = "<Search Text>"
assetType = "Volume"
ruleset = {
    "id": 123456,
    "name": "<Ruleset Name>",
    "vendor": "aws"
}

In [ ]:
filters = {
    "freeTextPhrase": str(text),
    "fields": [
        {
            "name": "entityType",
            "value": str(assetType)
        },
        {
            "name": "bundleId",
            "value": str(ruleset["id"])
        }
    ]
}

In [ ]:
searchByText = search(filters=filters)

In [ ]:
searchByText = [extractFindings(finding) for finding in searchByText]
searchByText = pd.DataFrame(searchByText)
searchByText

In [ ]:
nowTime = datetime.now().strftime("(%y%m%d-%H%M%S) Dome9_")
filepath = "outputs/" + nowTime + text + ".csv"

In [ ]:
searchByText.to_csv(filepath, index=False, encoding='utf-8')

## 2. Search Finding by Asset Name

In [ ]:
asset = {
    "name": "<Asset Name>",
    "type": "S3Bucket" # KubernetesNode
}
ruleset = {
    "id": 123456,
    "name": "<Ruleset Name>",
    "vendor": "aws"
}
cloudAccount = {
    "id": "<Cloud Account ID | GUID>",
    "name": "<Cloud Account Name>",
    "vendor": "aws"
}

In [ ]:
filters = {
    'freeTextPhrase': None,
    "fields": [
        {
            "name": "entityName",
            "value": str(asset["name"])
        },
        {
            "name": "entityType",
            "value": str(asset["type"])
        },
        {
            "name": "bundleId",
            "value": str(ruleset["id"])
        },
        {
            "name": "cloudAccountId",
            "value": str(cloudAccount["id"])
        }
    ]
}

In [ ]:
searchByAsset = search(filters=filters)

In [ ]:
searchByAsset = [extractFindings(finding) for finding in searchByAsset]

In [ ]:
searchByAsset = pd.DataFrame(searchByAsset)
searchByAsset